In [1]:
# repo imports
from main import run_backtest
from risk_manager import ExecParams

Briefly describe: MA + Trailing Stop, cooldown, min-hold, weekly exec, ATR catastrophic, giveback, time stop, vol targeting, gross-cap, rolling-CAGR filter.

In [2]:
# backtest parameters (feel free to adjust)
MA_WINDOW = 130
TRAIL_WINDOW = 10
TRAIL_PCT = 0.93
SHORT_THRESHOLD = 0.02
ATR_WIN = 14

COOLDOWN = 3
EPS = 0.005
MIN_HOLD = 5
WEEKLY = True
K_ATR = 2.5
GIVEBACK = 2.0
TIME_STOP = 60

TARGET_VOL = 0.10
GROSS_CAP = 1.0
ROLL_VOL = 20
PERF_LOOKBACK = 63

In [3]:
start_date = "2010-01-01"
end_date   = "2019-12-31"

stock_list = ["NVDA","MSFT","AAPL","AMZN","META","AVGO","GOOGL","TSLA","GOOG","BRK-B"]

In [4]:
results = run_backtest(
    start_date=start_date, end_date=end_date, stock_list=stock_list,
    ma_window=MA_WINDOW, trailing_window=TRAIL_WINDOW, trailing_pct=TRAIL_PCT,
    short_threshold=SHORT_THRESHOLD, atr_win=ATR_WIN,
    cooldown_bars=COOLDOWN, reentry_eps=EPS, min_hold=MIN_HOLD, weekly_exec=WEEKLY,
    k_atr_stop=K_ATR, giveback_atr=GIVEBACK, time_stop=TIME_STOP,
    target_vol=TARGET_VOL, gross_cap=GROSS_CAP, roll_vol=ROLL_VOL, perf_lookback=PERF_LOOKBACK,
    initial_capital=500_000.0, compare_spy=True, tag="final"
)
results["portfolio_metrics"]

[NOTIFY] Loading 10 symbols: NVDA, MSFT, AAPL, AMZN, META, AVGO, GOOGL, TSLA, GOOG, BRK-B


Failed to get ticker 'NVDA' reason: Expecting value: line 1 column 1 (char 0)
$NVDA: possibly delisted; No timezone found


ValueError: No data for NVDA in [2010-01-01, 2019-12-31]